## SONAR: Emotionally Intelligent Music Recommendations

Draws heavily from [this tutorial](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html)

Welcome to Sonar! This project was inpired me to first explore Machine Learning. The premise is simple: we often want different music to listen to based on our mood. However, oftentimes finding that music proves difficult. Sonar takes away that difficulty: by entering short mood messages, sonar generates Spotify recommendations using both your old favorites and determined mood. 

Enjoy!

In [26]:
import re

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool
import dill as pkl
import gensim
from gensim.models.word2vec import Word2Vec
from keras import Sequential
from keras.layers import Dense, Activation
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook, tqdm

nlp = spacy.load("en")
tqdm_notebook().pandas(desc="progress-bar")
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
TaggedDocument = gensim.models.doc2vec.TaggedDocument
STOPWORDS = set(stopwords.words('english'))
%matplotlib inline

### Preprocessing

- Load tweets into a usable format. Binarize sentiment from {0, 1, 2, 3, 4} to {0, 1} with 2 serving as the cut off for a "positive" sentiment. Clean up null values and reindex.

- Tokenize the data and take only the n values we aim to use in training.

In [2]:
def load_tweets():
    tweets_train = pd.read_csv("/home/malits/data/twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv", 
                               encoding = "ISO-8859-1")
    data = pd.DataFrame()
    data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v >= 2) else 0)
    data["sentiment"] = data["sentiment"].map(int)
    data["tweet"] = tweets_train.iloc[:, [5]]
    data["tweet"] = data["tweet"].astype(str)
    data = data[data['tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    
    return data

In [3]:
data = load_tweets()
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [4]:
def tokenize(tweet):
    sub_regex = r'(#[A-Za-z0-9])|(https)\?!\.,'
    tweet = str(tweet)
    tokens = tokenizer.tokenize(tweet)
    tokens = [re.sub(sub_regex, '', t) for t in tokens]
    tokens = [t.lower() for t in tokens if t is not None and t != '']
    tokens = [t for t in tokens if t not in STOPWORDS]
    return tokens

In [5]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data.loc[:, "tokens"] = data.loc[:, "tweet"].progress_apply(tokenize)
    data = data.loc[data.tokens != 'NC', :]
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [ ]:
processed = postprocess(data, len(data))

### Create training data

Once preprocessed, split to test/train sets and vectorize w w2vec for use in keras

In [7]:
# Size of our embedding space

N_DIM = 200

In [8]:
X_train, X_test, y_train, y_test = train_test_split(np.array(processed['tokens']),
                                                    np.array(processed['sentiment']),
                                                    test_size=0.2)

In [9]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i, v in tqdm_notebook(enumerate(tweets)):
        label = '{}_{}'.format(label_type, i)
        ## Use nltk TaggedDocument obj to mark words with their corresponding label. Makes words easier to retrieve from their vector form.
        labelized.append(TaggedDocument(v, [label]))
    return labelized

In [10]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')

## Plot

## Replace Word2Vec corpus with more data!

In [11]:
train_size = len(X_train)
tweet_w2v = Word2Vec(size=N_DIM, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm_notebook(X_train)])
tweet_w2v.train([x.words for x in tqdm_notebook(X_train)], 
                total_examples=train_size, epochs = tweet_w2v.epochs)

(43254461, 60140120)

In [12]:
def get_similarity(text):
    return tweet_w2v.wv.most_similar(text.lower())

In [13]:
get_similarity('kanye')

[('brom', 0.5770472288131714),
 ('indies', 0.5731891393661499),
 ('asher', 0.5657300353050232),
 ('abu', 0.5633035898208618),
 ('feat', 0.5593765377998352),
 ('cudi', 0.5551359057426453),
 ('nicholas', 0.5498791933059692),
 ('rascal', 0.5484213829040527),
 ('vida', 0.5476982593536377),
 ('fighters', 0.5446628332138062)]

In [14]:
keys = list(tweet_w2v.wv.vocab.keys())[:5000]

word_vectors = [tweet_w2v[w] for w in keys]

tsne_model = TSNE(n_components = 2, verbose = 1, random_state = 10)
tsne_w2v = tsne_model.fit_transform(word_vectors)

tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = keys

/home/malits/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.025s...
[t-SNE] Computed neighbors for 5000 samples in 8.214s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.561966
[t-SNE] KL divergence after 250 iterations with early exaggeration: 85.371964
[t-SNE] KL divergence after 1000 iterations: 2.369123


In [16]:
tsne_df.head(4)

,x,y,words
0,-41.065502,24.006413,awe
1,-1.834033,47.053192,","
2,-20.962540,41.751675,it's
3,-19.120577,-13.219883,not


In [22]:
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,"

p = figure(tools=TOOLS)

p.scatter(x='x', y='y', source=tsne_df)

hover = p.select(dict(type=HoverTool))
hover.tooltips = [
    ("word", "@words"),
]

output_notebook()

show(p)

Loading BokehJS ...

## Transforming

We use TF-IDF weightings to weight word vectors accordingly

In [28]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [29]:
def create_tweet_vector(words, ndims=200):
    vector = np.zeros((1, ndims))
    count = 0
    
    for w in words:
        try:
            vector += tweet_w2v.wv[w].reshape((1, ndims)) * tfidf[w]
            count += 1
        except KeyError:
            continue
    
    if count != 0:
        return np.divide(vector,count)
    else:
        return vector

In [30]:
train_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_train))])
train_vec_tweets = scale(train_vec_tweets)

test_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_test))])
test_vec_tweets = scale(test_vec_tweets)

## Neural Architecture in Keras

In [31]:
model = Sequential([
    Dense(32, input_dim=N_DIM),
    Activation('relu'),
    Dense(1),
    Activation('sigmoid')
])
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(train_vec_tweets, y_train, epochs=9, batch_size=32, verbose=2)

score = model.evaluate(test_vec_tweets, y_test, batch_size=32)

W0727 13:33:39.524584 139637494863680 deprecation_wrapper.py:119] From /home/malits/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/9
 - 25s - loss: 0.3493 - acc: 0.8545
Epoch 2/9
 - 25s - loss: 0.3369 - acc: 0.8595
Epoch 3/9
 - 25s - loss: 0.3338 - acc: 0.8610
Epoch 4/9
 - 25s - loss: 0.3318 - acc: 0.8623
Epoch 5/9
 - 25s - loss: 0.3305 - acc: 0.8626
Epoch 6/9
 - 25s - loss: 0.3297 - acc: 0.8630
Epoch 7/9
 - 25s - loss: 0.3289 - acc: 0.8632
Epoch 8/9
 - 25s - loss: 0.3283 - acc: 0.8637
Epoch 9/9
 - 25s - loss: 0.3280 - acc: 0.8640
200000/200000 [==============================] - 3s 13us/step


In [32]:
score

[0.3337647577881813, 0.861485]

In [67]:
def predict_text(texts):
    texts = [tokenize(text) for text in texts]
    matrix = np.concatenate([create_tweet_vector(t, N_DIM) for t in texts])
    return model.predict(matrix)

In [90]:
predict_text(["I hate everything. Today sucks!", "I love myself. Life is awesome!", "Make America Great Again", "Kanye West", "Caleb Litster"])

array([[7.7025041e-05],
       [9.9925810e-01],
       [9.9969125e-01],
       [9.9715638e-01],
       [9.8254794e-01]], dtype=float32)

## Pickling the neural model, w2vec model and tfidf lookup

In [34]:
"""
pkl.dump(model, open("data/sentiment_classifier00.pkl", "wb"))
pkl.dump(tweet_w2v, open("data/tweet_w2v00.pkl", "wb"))
pkl.dump(tfidf, open("data/tfidf_lookup00.pkl", "wb"))
"""

'\npkl.dump(model, open("data/sentiment_classifier00.pkl", "wb"))\npkl.dump(tweet_w2v, open("data/tweet_w2v00.pkl", "wb"))\npkl.dump(tfidf, open("data/tfidf_lookup00.pkl", "wb"))\n'

## Productionalizing Code

In [69]:
class SentimentAnalyzer:
    
    def __init__(self):
        self.ndims = 200
    
    def load_model(self):
        self.classifier = pkl.load(open("data/sentiment_classifier00.pkl", "rb"))
        self.word2vec = pkl.load(open("data/tweet_w2v00.pkl", "rb"))
        self.tfidf = pkl.load(open("data/tfidf_lookup00.pkl", "rb"))
        
    def tokenize(self, tweet):
        sub_regex = r'(#[A-Za-z0-9])|(https)\?!\.,'
        tweet = str(tweet)
        tokens = tokenizer.tokenize(tweet)
        tokens = [re.sub(sub_regex, '', t) for t in tokens]
        tokens = [t.lower() for t in tokens if t is not None and t != '']
        tokens = [t for t in tokens if t not in STOPWORDS]
        return tokens
    
    def create_tweet_vector(self, words):
        vector = np.zeros((1, self.ndims))
        count = 0

        for w in words:
            try:
                vector += self.tweet_w2v.wv[w].reshape((1, self.ndims)) * self.tfidf[w]
                count += 1
            except KeyError:
                continue

        if count != 0:
            return np.divide(vector,count)
        else:
            return vector
    
    def predict_sentiment(self, texts):
        texts = [self.tokenize(text) for text in texts]
        tweet_vectors = np.concatenate([self.create_tweet_vector(text) for text in texts])
        return self.classifier.predict(tweet_vectors)